## Library Widgets for Graph Exploration
In this early example, we are demonstrating the ability to populate an initial LPG graph (networkx) using the `GraphExporeNodeSelection` widget. This widget uses a custom `SelectMultipleURI` UI widget in order to display pithy URIs, while tracking URIRefs (via `rdflib`). The type selection passes the URI values to the subject select portion of the widget; this uses a [metaclass to update a SPARQL query](SPARQL_Metaclass_Queries.ipynb) in order to populate options.

In a future update, the `GraphExploreNodeSelection` widget will be connected directly to the LPG graph visualization widget. 

### Load an RDF graph

In this example, we will use the `ipyradiant` `FileManager`.

In [ ]:
from ipyradiant import FileManager, PathLoader

lw = FileManager(loader=PathLoader(path="data"))
# here we hard set what we want the file to be, but ideally a user can choose a file to work with.
lw.loader.file_picker.value = lw.loader.file_picker.options["starwars.ttl"]
lw

### GraphExploreNodeSelection
This widget allows the user to select Nodes to populate an initial LPG graph. The nodes are first filtered by their `rdf:type`, and then selected from the subjects available in the loaded graph.

In [ ]:
from ipyradiant.visualization.explore import GraphExploreNodeSelection

ge_selector = GraphExploreNodeSelection()
ge_selector.graph = lw.graph
ge_selector

For the example, we will automatically select: 
1. Available types: `voc:Droid`, `voc:Film`
2. Available subjects: `A New Hope`, `C-3PO`, `R2-D2`

In [ ]:
from rdflib import URIRef

# this sets our selection in the widget so that we don't have to click manually
ge_selector.type_select.select_widget.value = (
    URIRef('https://swapi.co/vocabulary/Droid'),
    URIRef('https://swapi.co/vocabulary/Film')
)
ge_selector.subject_select.select_widget.value = (
    URIRef('https://swapi.co/resource/film/1'),
    URIRef('https://swapi.co/resource/droid/2'),
    URIRef('https://swapi.co/resource/droid/3')
)

In [ ]:
# Run this to show the selected values from the subject select widget
ge_selector.subject_select.select_widget.value

The following cell shows how you can use the SelectMultipleURI method `get_pithy_uri` to return the (e.g. `CustomURI`) URI classes for the selected subjects.

In [ ]:
ssw = ge_selector.subject_select.select_widget
tuple(map(ssw.get_pithy_uri, ssw.value))

Let's make sure that the selections are what we expect.

In [ ]:
EXPECTED_SELECTIONS = {
    URIRef('https://swapi.co/resource/droid/2'),
    URIRef('https://swapi.co/resource/film/1'),
    URIRef('https://swapi.co/resource/droid/3'),
}
assert set(ge_selector.subject_select.select_widget.value) == EXPECTED_SELECTIONS, \
    "Make sure to select the correct items for the example."

### Pass to RDF2NX
The remainder of this notebook is just to illustrate that we can pass the selections to the [RDF2NX](RDF_to_NX.ipynb) class in order to generate an LPG for the nodes. 

In [ ]:
from ipyradiant.rdf2nx import RDF2NX

nx_graph = RDF2NX.convert_nodes(node_uris=ge_selector.subject_select.select_widget.value, rdf_graph=lw.graph)

In [ ]:
import json

import ipycytoscape
import ipywidgets as W

In [ ]:
directed = ipycytoscape.CytoscapeWidget()
directed.graph.add_graph_from_networkx(nx_graph, multiple_edges=True, directed=True)

In [ ]:
for node in directed.graph.nodes:
    # deal with cytoscape's inability to handle `:` e.g. thing:data
    node.data["_label"] = node.data.get("rdfs:label", None)
    node.data["_attrs"] = json.dumps(node.data, indent=2)

In [ ]:
directed.set_layout(
    name="dagre", animate=False, randomize=False, maxSimulationTime=2000
)
# Workaround for style overwriting
directed.set_style(
    [
        {
            "selector": "node",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "blue",
            },
        },
        {
            "selector": "edge",
            "css": {
                "label": "data(_label)",
                "text-wrap": "wrap",
                "text-max-width": "150px",
                "text-valign": "center",
                "text-halign": "center",
                "font-size": "10",
                "font-family": '"Gill Sans", sans-serif',
                "color": "green",
            },
        },
        {
            "selector": "edge.directed",
            "style": {
                "curve-style": "bezier",
                "target-arrow-shape": "triangle",
            },
        },
        {"selector": "edge.multiple_edges", "style": {"curve-style": "bezier"}},
        {
            "selector": ":active ",
            "css": {
                "label": "data(_attrs)",
                "text-wrap": "wrap",
                "text-max-width": "500px",
                "text-valign": "bottom",
                "text-halign": "right",
                "text-background-opacity": 0.9,
                "text-background-color": "white",
                "text-background-shape": "roundrectangle",
                "color": "black",
            },
        },
    ]
)

The directed graph is just a basic example of how the LPG can be visualized. 

Clicking the node (hold down mouse click) allows for basic inspection of the node properties. 

In [ ]:
directed